In [1]:
import pandas as pd
import os, re
import statistics as sts

In [2]:
#IMPORTAMOS LOS GENES QUE NOS INTERESAN

genes1 = pd.read_csv('../Genes/genes-metabolismo.csv', header = None)
genes2 = pd.read_csv('../Genes/genes-metabolismo.csv', header = None)
genes3 = pd.read_excel('../Genes/genes2Nuevo.xlsx',header = None)
genes = set(genes1[0]).union(set(genes2[0])).union(genes3[0])

In [3]:
datas_name = "GSE155974"
folder = f"../data_sources/New/{datas_name}/"
data = pd.DataFrame()
for folder_name in os.listdir(folder):
    folder_path = f'{folder}/{folder_name}'
    fn = f"{folder_path}/{os.listdir(folder_path)[0]}"
    new_data = pd.read_csv(fn, sep="\t")
    
    if data.empty:
            data.index = new_data["gene_id"]
    
    match = re.search(r"((24|48)h)_(replicate(1|2))", folder_name)
    if match:
        col_name = f"{match.group(3)} {match.group(1)}"
    else:
        col_name = "mock"
    data[col_name] = list(new_data[new_data.columns[1]])
        
data

,replicate1 24h,replicate2 24h,replicate1 48h,replicate2 48h,mock
gene_id,,,,,
A1BG,42,19,29,22,2
A1CF,1,2,2,2,0
A2M,51,68,48,38,71
A2ML1,494,509,147,458,149
A3GALT2,0,0,2,0,0
...,...,...,...,...,...
ZYG11A,36,60,25,24,27
ZYG11B,682,1105,491,501,1205
ZYX,1234,790,859,927,817


In [4]:
for infection_time in ["24h", "48h", "mock"]:
    replicates = data.filter(like=infection_time)
    data[f"{infection_time} mean"] = pd.DataFrame(replicates.mean(axis=1))
# data = data.drop(data.filter(like="replicate"), axis=1)
# data

In [5]:
means = data.filter(like="mean")
for GAPDH,Columna in zip(list(means.loc['GAPDH']),list(means)) :
    data[Columna.replace("mean", "")+"norm"] = data[Columna]/GAPDH
# data = data.drop(data.filter(like="mean"), axis=1)
# data

In [6]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))
from Funciones import fold_change

In [7]:
data.columns

Index(['replicate1 24h', 'replicate2 24h', 'replicate1 48h', 'replicate2 48h',
       'mock', '24h mean', '48h mean', 'mock mean', '24h norm', '48h norm',
       'mock norm'],
      dtype='object')

In [8]:
data[f"fold-change 24h"] = fold_change(data["mock"], data["24h norm"])
data[f"fold-change 48h"] = fold_change(data["mock"], data["48h norm"])
data

,replicate1 24h,replicate2 24h,replicate1 48h,replicate2 48h,mock,24h mean,48h mean,mock mean,24h norm,48h norm,mock norm,fold-change 24h,fold-change 48h
gene_id,,,,,,,,,,,,,
A1BG,42,19,29,22,2,30.5,25.5,2.0,0.000865,0.000684,0.000113,-11.174762,-11.513882
A1CF,1,2,2,2,0,1.5,2.0,0.0,0.000043,0.000054,0.000000,inf,inf
A2M,51,68,48,38,71,59.5,43.0,71.0,0.001688,0.001153,0.004006,-15.360429,-15.90979
A2ML1,494,509,147,458,149,501.5,302.5,149.0,0.014225,0.008113,0.008407,-13.354562,-14.164685
A3GALT2,0,0,2,0,0,0.0,1.0,0.0,0.000000,0.000027,0.000000,0,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,36,60,25,24,27,48.0,24.5,27.0,0.001362,0.000657,0.001523,-14.275424,-15.326485
ZYG11B,682,1105,491,501,1205,893.5,496.0,1205.0,0.025345,0.013303,0.067987,-15.536993,-16.466929
ZYX,1234,790,859,927,817,1012.0,893.0,817.0,0.028706,0.023951,0.046096,-14.796698,-15.057984


In [9]:
#DF fold-change
fold_change_data = data.filter(like='fold-change')

In [10]:
missing_genes = genes-set(fold_change_data.index)
fold_change_data = pd.DataFrame(fold_change_data, index=list(fold_change_data.index)+list(missing_genes), columns=fold_change_data.columns)

In [11]:
output_dir = f"../fold-change/{datas_name}"
os.makedirs(output_dir, exist_ok=True)
for indx, gene in enumerate([genes1, genes2, genes3]):
    fold_change_data.loc[list(gene.loc[:,0])].to_csv(f"{output_dir}/fold_change_genes_{indx+1}_{datas_name}.csv")

output_dir = f"../Resultados/{datas_name}"
os.makedirs(output_dir, exist_ok=True)
data.to_csv(f'{output_dir}/Resultados_Analisis_{datas_name}.csv')

In [12]:
fold_change_data.loc[list(genes2.loc[:,0])]

,fold-change 24h,fold-change 48h
AKR1B1,-15.36125,-15.611984
ALDH4A1,-12.363493,-12.79399
CBR1,-15.351373,-15.560305
COMT,-14.730852,-14.837203
DBH,inf,inf
...,...,...
NUDT16,-14.209827,-14.838866
SUOX,-14.547232,-14.993482
THY1,-14.1795,-14.115918
TPMT,-15.366619,-15.857929
